# Download Active Listings
Etsy API limits the offset to 12,000 listings, so that is the maximum number of recently updated listings that can be accessed. (Requested 100 at a time because each request is limited to 100 records)

In [1]:
import requests
import pandas as pd
import json

Get all listings that match keywords "stoneware" and "handmade"

In [2]:
#GetListings URL and Parameters for the API
url = "https://openapi.etsy.com/v3/application/listings/active"
key = "r8ymjsnfohuidr1wev7e4cfg"
keyword = "stoneware,handmade"

In [3]:
#initialize the dataframes
dataframe = pd.DataFrame()
df = pd.DataFrame()

In [4]:
#loop that increases the offset by 100 each time due to the API limitation of downloading a maximum of 100 listings at a time
limit = 100
offset = 0
for i in range (1,121):
    parameters = {"client_id":key, "limit":limit, "offset":offset, "keywords":keyword}
    download = requests.get(url, params=parameters)
    data = download.json()
    if "error" in data.keys():
        print("Maximum Offset Reached")
    else:
        dataframe = pd.json_normalize(data['results'])
        df = df.append(dataframe, ignore_index = True)
    offset = offset + limit
    i = i+1
    

In [5]:
#confirm that there are 12,000 rows
len(df)

12000

In [6]:
#OPTIONAL
#write the data to a csv file
df.to_csv('listingsStonewareHandmade041022.csv')

In [7]:
df.columns


Index(['listing_id', 'user_id', 'shop_id', 'title', 'description', 'state',
       'creation_timestamp', 'ending_timestamp', 'original_creation_timestamp',
       'last_modified_timestamp', 'state_timestamp', 'quantity',
       'shop_section_id', 'featured_rank', 'url', 'num_favorers',
       'non_taxable', 'is_customizable', 'is_personalizable',
       'personalization_is_required', 'personalization_char_count_max',
       'personalization_instructions', 'listing_type', 'tags', 'materials',
       'shipping_profile_id', 'processing_min', 'processing_max', 'who_made',
       'when_made', 'is_supply', 'item_weight', 'item_weight_unit',
       'item_length', 'item_width', 'item_height', 'item_dimensions_unit',
       'is_private', 'style', 'file_data', 'has_variations',
       'should_auto_renew', 'language', 'taxonomy_id', 'production_partners',
       'skus', 'price.amount', 'price.divisor', 'price.currency_code'],
      dtype='object')

## Download Shop Info for all the Etsy listings in the data



In [8]:
#NOTE: DELETE LATER You dont need this if you are doing downloading the 12,000 listings and the shop info at the same time
#df = pd.read_csv('12000listingsStonewareHandmade032222.csv')

In [9]:
#Get all the unique shop IDs
shop_ids = df["shop_id"].tolist()
shop_list = list(set(shop_ids))

In [10]:
? shop_list

In [11]:
#initialize the dataframes
dataframeShop = pd.DataFrame()
dfShop = pd.DataFrame()

# Get Shop Info
This loop sends a request to the ETSY API to get the shop info (ratings, number of favorers, etc) for each shop that appeared in the previous dataset

In [12]:
#loop that queries the API for 
limit = 100

for j in range(0, len(shop_list)):
    shop_id = str(shop_list[j])
    url = "https://openapi.etsy.com/v3/application/shops/" + shop_id
    parameters = {"client_id":key, "limit":limit}
    download = requests.get(url, params=parameters)
    dataShops = download.json()
    dataframeShop = pd.json_normalize(dataShops)
    dfShop = dfShop.append(dataframeShop, ignore_index = True)

In [13]:
dfShop

,shop_id,shop_name,user_id,create_date,title,announcement,currency_code,is_vacation,vacation_message,sale_message,...,policy_shipping,policy_refunds,policy_additional,policy_seller_info,policy_update_date,policy_has_private_receipt_info,has_unstructured_policies,policy_privacy,review_average,review_count
0,6676480,GaiasSacredCreations,9442907,1325095539,Handmade Ceramics Porcelain Pottery Glass,https://www.gaiassacredcreations.com/ \r\n U...,USD,False,None,Thank you for purchasing my work! \r\n\r\nPLEA...,...,I use Canada Post Airmail first Class for smal...,I will refund (within 30 days of purchase) th...,This Privacy Policy describes how and when I c...,None,1590239891,True,True,None,5.0,42
1,7266306,MalenkaDesign,24635635,1347577309,Handcrafted fonctional pottery from Santa Moni...,instagram: malenkadesign_ceramics\r\n\r\nI loo...,USD,False,None,None,...,I ship via USPS First Class Mail. \r\n\r\n****...,In the unlikely event your pottery is damaged ...,If you are sending your purchase as a gift for...,None,1423758137,False,True,None,5.0,10
2,26124291,FoxCreekCeramics,317688587,1605750569,Hand-thrown ceramics in stoneware and English ...,Welcome to my site. I am a small boutique pot...,USD,False,Hello and thanks for visiting! I’m working on ...,Thanks so much for making a purchase from Fox ...,...,PROCESSING TIME\nThe time I need to prepare an...,I GLADLY ACCEPT RETURNS\nJust contact me withi...,- I do not share your contact information with...,None,0,False,False,I do not share your contact information with a...,5.0,1
3,23371778,KeramikTonstudio,307866980,1590185007,"originelle Deko aus Ton, für Haus und Garten",Herzlich willkommen im KeramikTonstudio.\nIhr ...,EUR,False,Allen Kundinnen und Kunden frohe Festtage und ...,None,...,PROCESSING TIME\nThe time I need to prepare an...,I GLADLY ACCEPT RETURNS AND EXCHANGES\nJust co...,,None,0,False,False,None,5.0,66
4,19709955,EversArtShop,73391987,1553216295,Spreading love through art!,Hello my friends and welcome to EversArtShop w...,USD,False,None,None,...,PROCESSING TIME\nThe time I need to prepare an...,"I GLADLY ACCEPT RETURNS, EXCHANGES, AND CANCEL...",,None,0,False,False,,4.9545,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2862,20709362,SamNorlinCeramics,229058406,1568752211,None,None,USD,False,None,None,...,None,None,None,None,0,False,False,None,5,3
2863,24084468,MeganHawthorneStudio,329705074,1598723493,Handmade Ceramics and Homewares,Hi there and welcome to my Etsy shop! Have a l...,GBP,False,None,None,...,None,None,None,None,0,False,False,None,5,12
2864,28565493,VivaLaVulvaCeramics,451129385,1616418862,vulva-shaped functional ceramic mugs and pipes,Viva La Vulva!\nOur aim is to destroy the tabo...,TRY,False,None,None,...,PROCESSING TIME\nThe time I need to prepare an...,I GLADLY ACCEPT RETURNS AND EXCHANGES\nJust co...,- This Privacy Policy describes how and when I...,None,0,False,False,This Privacy Policy describes how and when I c...,5,9
2865,32096248,LoveoftheClay,535595918,1646001865,Handmade Stoneware Pottery,FREE SHIPPING on orders $35 or more\nSPRING SO...,USD,False,None,None,...,PROCESSING TIME\nThe time I need to prepare an...,I GLADLY ACCEPT CANCELLATIONS\nRequest a cance...,IS THE POTTERY FOOD-SAFE?\nYES. All glazes use...,None,0,False,False,,5,5


In [14]:
#OPTIONAL
#write the data to a csv file
dfShop.to_csv('ShopInfo041022.csv')

## Merge the data

In [15]:
#create a dataframe with just the columns I'm interested in for the listings data
listings = df[['listing_id', 'shop_id', 'title',
       'description', 'num_favorers',
       'listing_type', 'tags', 'materials',
       'style','taxonomy_id','price.amount',
       'price.divisor', 'price.currency_code','creation_timestamp', 'ending_timestamp',
       'original_creation_timestamp','quantity']]

In [16]:
#create a dataframe with just the columns I'm interested in for the shops data
shops = dfShop[['shop_id', 'shop_name', 'create_date', 'title',
       'announcement', 'currency_code', 'is_vacation', 'vacation_message',
       'sale_message',
       'listing_active_count',
       'accepts_custom_requests', 'url',
       'num_favorers', 'languages', 'is_shop_us_based', 'transaction_sold_count',
       'shipping_from_country_iso', 'shop_location_country_iso',
       'review_average', 'review_count']]

In [17]:
#merge the two datasets so that the shop information is added to each listing
data = listings.merge(shops, on="shop_id")

In [18]:
# save the merged data
#NOTE: change the date each time (maybe automate the filename with the correct date for the future)

data.to_csv('mergedData041022.csv')